In [1]:
import os, gc
import numpy as np
import pandas as pd
from keras import backend as K
from keras.layers import Input, MaxPooling2D, UpSampling2D, Conv2D
#from keras.layers import concatenate
#from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose
from keras.models import Model
from keras.optimizers import Adam
from skimage.transform import resize
from skimage.io import imsave, imread
from segmodels import dice_coef, dice_coef_loss, double_conv_layer
from segmodels import create_model, preprocess_img, preprocess, test_generator
from keras.callbacks import ModelCheckpoint
from matplotlib import pyplot as plt
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout, Activation
from keras import backend as K
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline 

smooth = 1.

gc.collect()

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


1164

In [2]:
img_rows = 512
img_cols = 512
batch_size = 4
nb_epoch = 10
print(img_rows, img_cols)
data_path = '/home/ubuntu/noaa/darknet/seals/'
train_path = 'JPEGImagesBlk'
test_path  = 'JPEGImagesTest'
train_mask_path = '../data/mask/all/'
pred_dir = '../data/mask/all_pred/'
smooth = 1.
K.image_dim_ordering()

(512, 512)


'th'

In [ ]:
def train_data(path, exclude = []):
    images = os.listdir(train_mask_path)
    images = [i for i in images if i not in exclude]
    total = len(images)
    imgs = np.ndarray((total, img_rows, img_cols, 3), dtype=np.float32)
    imgs_mask = np.ndarray((total, img_rows, img_cols), dtype=np.uint8)

    i = 0
    print('-'*30)
    print('Creating training images...')
    print('-'*30)
    for image_mask_name in images:
        image_name = image_mask_name.split('.')[0] + '.jpg'
        #image_mask_name = image_name.split('.')[0] + '_mask.tif'
        img = imread(os.path.join(data_path, train_path, image_name), as_grey=False)
        img_mask = imread(os.path.join(train_mask_path, image_mask_name), as_grey=True)
        img = resize(img, (img_rows, img_cols), mode='reflect')
        img_mask = resize(img_mask, (img_rows, img_cols), mode='reflect')

        img = np.array([img])
        img_mask = np.array([img_mask])

        imgs[i] = img
        imgs_mask[i] = img_mask
        if i % 500 == 0:
            print('Done: {0}/{1} images'.format(i, total))
        i += 1
    print('Loading done.')
    return imgs, imgs_mask

def test_generator(df, batch_size = 4):
    n = df.shape[0]
    batch_index = 0
    while 1:
        current_index = batch_index * batch_size
        if n >= current_index + batch_size:
            current_batch_size = batch_size
            batch_index += 1    
        else:
            current_batch_size = n - current_index
            batch_index = 0        
        batch_df = df[current_index:current_index+current_batch_size]
        batch_x = np.zeros((batch_df.shape[0], img_rows, img_cols, 3)).astype('float32')
        i = 0
        for index,row in batch_df.iterrows():
            img = imread(os.path.join(data_path, test_path, row[0]), as_grey=False)
            img = resize(img, (img_rows, img_cols), mode='reflect')
            x = np.array([img])
            x -= mean
            x /= std
            batch_x[i] = x
            i += 1
        if batch_index%300 == 0: print(batch_index)
        yield(batch_x.transpose(0, 3, 1, 2))

In [ ]:
for fold in range(1,2):
    print('-'*30)
    print('Directory structure...')
    print('-'*30)
    print(train_path , test_path , pred_dir )
    
    # exclude any not in that fold
    images = os.listdir(train_mask_path)
    images = [i for i in images if '.tif' in i]
    exclude = [i for i in images if int(i.split('_')[0])%2==fold]
    
    imgs_train, imgs_mask_train = train_data(train_path, exclude)
    imgs_train = preprocess_img(imgs_train, img_rows, img_cols, 3)
    imgs_mask_train = preprocess(imgs_mask_train, img_rows, img_cols, 1)

    imgs_train = imgs_train.astype('float32')
    mean = np.mean(imgs_train)  # mean for data centering
    std = np.std(imgs_train)  # std for data normalization

    imgs_train -= mean
    imgs_train /= std

    imgs_mask_train = imgs_mask_train.astype('float32')

    imgs_train = imgs_train[:,:,:,:,0]
    imgs_mask_train = imgs_mask_train[:,:,:,:,0]

    #del model
    print('-'*30)
    print('Creating and compiling model...')
    print('-'*30)
    model = create_model(img_rows, img_cols, 3)
    model_checkpoint = ModelCheckpoint('weights_seg.h5', monitor='val_loss', save_best_only=True)

    print('-'*30)
    print('Fitting model...')
    print('-'*30)
    optim = Adam(lr=.001)
    model.compile(optimizer=optim, loss=dice_coef_loss, metrics=[dice_coef])
    model.fit(imgs_train.transpose(0, 3, 1, 2), imgs_mask_train.transpose(0, 3, 1, 2), batch_size=batch_size, 
              verbose=1, shuffle=True, nb_epoch=nb_epoch,
              validation_split=0.1,
              callbacks=[model_checkpoint])

    print('-'*30)
    print('Predicting test...')
    print('-'*30)
    test_df = [f.split('.')[0]+'.jpg' for f in exclude]
    test_df = pd.DataFrame(test_df)
    imgs_mask_test = model.predict_generator(test_generator(test_df),  
                                             val_samples = test_df.shape[0])
    imgs_mask_test = imgs_mask_test.transpose(0, 2, 3, 1)


    print('-' * 30)
    print('Saving predicted masks to files...')
    print('-' * 30)
    if not os.path.exists(pred_dir):
        os.mkdir(pred_dir)
    for i in range(len(imgs_mask_test)):
        fname = test_df[0][i].split('.')[0]
        imsave(os.path.join(pred_dir, str(fname) + '_pred.png'), imgs_mask_test[i][:,:,0])
        
    del imgs_train, imgs_mask_train, imgs_mask_test
    gc.collect()
    gc.collect()

------------------------------
Directory structure...
------------------------------
('JPEGImagesBlk', 'JPEGImagesTest', '../data/mask/all_pred/')
------------------------------
Creating training images...
------------------------------
Done: 0/3755 images
Done: 500/3755 images
Done: 1000/3755 images
